In [1]:
import logging
import numpy as np
import pandas as pd
import anndata as ad
from sklearn.impute import KNNImputer
import pickle

from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.decomposition import TruncatedSVD

/Users/ishitakokil/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load AnnData objects
train_adt = ad.read_h5ad("Adt_processed_training.h5ad")
test_adt =  ad.read_h5ad("Adt_processed_testing.h5ad")
test_gex = ad.read_h5ad("Gex_processed_testing.h5ad")
train_gex = ad.read_h5ad("Gex_processed_training.h5ad")

# Impute missing values using KNN imputation
imputer = KNNImputer(n_neighbors=5)
train_gex.X = imputer.fit_transform(train_gex.X.toarray())
test_gex.X = imputer.transform(test_gex.X.toarray())

with open("train_gex_imputed.pkl", "wb") as f:
    pickle.dump(train_gex, f)
with open("test_gex_imputed.pkl", "wb") as f:
    pickle.dump(test_gex, f)

In [3]:
# Load the pickled file   
with open("train_gex_imputed.pkl", "rb") as f:
    train_gex = pickle.load(f)
    
with open("test_gex_imputed.pkl", "rb") as f:
    test_gex= pickle.load(f)

In [4]:
# This will get passed to the method
input_train_gex = train_gex
input_train_adt = train_adt
input_test_gex =  test_gex

true_test_adt =  test_adt

In [7]:
import numpy as np
import logging
import anndata as ad
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPRegressor
import pickle
from sklearn.metrics import mean_squared_error

def train_and_save_model(input_train_gex, input_train_adt):
    input_gex = input_train_gex

    # Perform dimensionality reduction on GEX values
    logging.info("Performing dimensionality reduction on GEX values...")
    embedder_gex = TruncatedSVD(n_components=50)
    gex_pca = embedder_gex.fit_transform(input_gex.X)

    # Prepare training data
    X_train = gex_pca
    y_train = input_train_adt.X.toarray()

    # Build and train a MLP regressor model
    reg = MLPRegressor(
        hidden_layer_sizes=(50, 50),
        max_iter=1000,
        random_state=0,
    )
    reg.fit(X_train, y_train)

    # Save the trained model and the PCA embedding as pickle files
    with open("mlp_model_ishita.pkl", "wb") as f:
        pickle.dump(reg, f)
    with open("pca_embedding_ishita.pkl", "wb") as f:
        pickle.dump(embedder_gex, f)


In [ ]:
train_and_save_model(input_train_gex,input_train_adt)

In [ ]:
def ishita_model(test_gex):
    # Load the trained model and PCA embedding from the pickle files
    with open("mlp_model_ishita.pkl", "rb") as f:
        trained_model = pickle.load(f)
        
    with open("pca_embedding_ishita.pkl", "rb") as f:
        pca_embedding = pickle.load(f)
        
    imputer = KNNImputer(n_neighbors=5)
    test_gex.X = imputer.fit_transform(test_gex.X)
        
    # Perform dimensionality reduction on the input_test_gex
    X_test = pca_embedding.transform(test_gex.X)

    y_pred = trained_model.predict(X_test)
    
    # Project the predictions back to the adt feature space
    pred_test_adt = ad.AnnData(
        X = np.array(y_pred),
        obs = input_test_gex.obs,
        var = input_train_adt.var,
    )
    return pred_test_adt
    

In [ ]:
pred_test_adt = ishita_model(input_test_gex)

In [ ]:
mlp_rmse=mean_squared_error(true_test_adt.X.toarray(), pred_test_adt.X, squared=False)

In [ ]:
print(mlp_rmse)